<a href="https://colab.research.google.com/github/CGame1/Img_classificaton_guide/blob/main/notebooks/create_hf_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
from datasets import load_dataset
import os

#path to image folder
path = "../images"

#load as dataset
dataset = load_dataset("imagefolder", data_dir=path, split="all")

In [ ]:
import numpy as np
import pandas as pd
import os


# load annotations
data = pd.read_csv("../Annotations.csv")

# remove non-image rows
data.dropna(subset=["IMAGE water"], inplace=True)

# only use image rows
labels = list(data["Cluster descr (Linktree Clusters)"])

# get full image paths
imgs = list(path + data["IMAGE water"])

# only work on existing images
idx = [os.path.exists(img) for img in imgs]
labels = list(np.array(labels)[idx])
imgs = list(np.array(imgs)[idx])

labels_dict = {os.path.basename(img): label for img, label in zip(imgs, labels)}

labels_dict


In [ ]:
#add labels to dataset
def add_labels(example):
    fname = os.path.basename(example["image"].filename)
    example["label"] = labels_dict[fname]

    example["filename"] = os.path.basename(example["image"].filename)   
     
    return example

dataset = dataset.map(add_labels)


In [ ]:
#pust to hugging face
dataset.push_to_hub("CGame1/schulz_bank_biotopes", split='full')

In [ ]:
from huggingface_hub import login
login()

In [ ]:
#load dataset from hugging face
from datasets import load_dataset
name = "CGame1/schulz_bank_biotopes"
dataset = load_dataset(path=name, split="all")


In [ ]:
# save images locally
from pathlib import Path
import numpy as np
import os
from tqdm import tqdm 

base_path = Path("../images")

for i in tqdm(np.arange(len(dataset))):
    i = int(i)
    save_path = base_path

    if not os.path.exists(save_path):
        os.mkdir(save_path)

    dataset[i]["image"].save(f"{save_path}/{dataset[i]["filename"]}")